# Coleta de dados na Web

* As marés dos próximos sete dias: Coletando informaçoes sobre as condições das marés em Tadoussac para os próximos sete dias a partir de 21 de setembro de 2023.

## Importando bibliotecas Python

In [3]:
# Importação das libs
#-*- coding: utf-8 -*-

from bs4 import BeautifulSoup #Biblioteca de manipulação HTML

#edgeDriver ou outro para controlar o navegador com Selenium, 
#dependendo do navegador usado
from selenium import webdriver 

import re
import pandas as pd

print("Bibliotecas Python importadas")

Bibliotecas Python importadas


## Análisando os pontos de entrada de formulário com a função de inspeção do browser:
<ul>
    <ul>
        <li>Abrir a página do <a href="https://www.marees.gc.ca/fra/station?sid=2985" target='_blank'>formulário</a> no Web site Pêches et Océans Canada;</li>
        <li>Reconstruir a sequência de interações com o formulário (cliques, menus e entrada do teclado);</li>
        <li>Obter identificadores de botões, menus, campos de entrada;</li>
    </ul>
</ul>   

## Programando um script de interação Selenium com o formulário das marés



In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from warnings import filterwarnings
filterwarnings("ignore")


#Instanciar um driver para o navegador Chrome/msEdge - método obsoleto
#robot_edge = webdriver.Edge("msedgedriver.exe")

#Nova maneira de instalar o driver para o navegador Edge
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
robot_edge = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))


#Abrindo a página Web onde o formulário das marés está localizado
robot_edge.get("https://www.marees.gc.ca/fr/stations/")

localidade = "Tadoussac"

#Solicitar o formulário para a localidade de Tadoussac com a ferramenta de pesquisa / filtragem
seizure_localidade = robot_edge.find_element(By.XPATH, "//div/label/input")
seizure_localidade.clear()
seizure_localidade.send_keys(localidade)

#Solicitar um tempo de resposta
import time 
time.sleep(2)

#Clique no hiperlink para a página web de marés de Tadoussac
hiperlink_localidade = robot_edge.find_element(By.XPATH, "//td/a")
hiperlink_localidade.click()

#Solicitar um tempo de resposta
time.sleep(2)

#Definindo o componente calendário, para 14 de setembro de 2023
seizure_date = robot_edge.find_element(By.ID, "water-levels-date")
seizure_date.clear()
robot_edge.execute_script("arguments[0].value = '2023-09-14';", seizure_date)
#Enviar o pedido para obter o cronograma de maré em Tadoussac para
# os sete dias de procahins a partir de 14 de setembro de 2023
bouton_send = robot_edge.find_element(By.ID, "water-levels-date-submit")
bouton_send.click()

#Solicitar um tempo de resposta
time.sleep(2)

#Obtendo os resultados da página Web
page_answer = robot_edge.page_source

print("Selenium script deve abrir um navegador e exibir uma mensagem")
print("do género: « Browser is being controlled by automated test software. »")
print("em seguida, o script Selenium simula interações com o formulário.\n")
print("Script Selenium executado, formulário enviado, página de resultados recuperada...")

Selenium script deve abrir um navegador e exibir uma mensagem
do género: « Browser is being controlled by automated test software. »
em seguida, o script Selenium simula interações com o formulário.

Script Selenium executado, formulário enviado, página de resultados recuperada...


## Análise dos resultados retornado da página web  com `BeautifulSoup`

### Exibição formatada da página web de resultados em HTML 

In [5]:
#Análise dos resultados retornado da página web com BeautifulSoup
page_resultats = BeautifulSoup(page_answer,"html.parser") 

#Inspecionando a página Web devolvida
print(page_resultats.prettify())

<html class="js js backgroundsize borderimage csstransitions fontface svg details progressbar meter mathml cors mediumview wb-enable" dir="ltr" lang="fr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <script src="https://www.marees.gc.ca/libraries/wet-boew/js/i18n/fr.min.js">
  </script>
  <script src="https://www.marees.gc.ca/libraries/wet-boew/js/deps/jquery.dataTables.min.js">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-W9G4T7DH4P&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script async="" src="https://www.google-analytics.com/analytics.js">
  </script>
 

##  Extraindo seções relevantes da página de resultados com `Beautiful Soup`

### Visualização das secções relevantes que contêm dados das marés durante sete dias

In [8]:
#Recuperando parâmetros da consulta de pesquisa de informações
raw_parameters = page_resultats.find_all("table", {"class":"day-tables clearfix table-bordered responsive-enabled table table-hover table-striped"})
print("Seções relevantes em HTML:\n")
print("Número de dias:",len(raw_parameters),"\n")

#raw_parameters = raw_parameters.find("div", {"class":"wb-eqht wb-init wb-eqht-inited"})
print(raw_parameters)

Seções relevantes em HTML:

Número de dias: 7 

[<table aria-label="Times and Heights for High and Low Tides for 2023-09-14" class="day-tables clearfix table-bordered responsive-enabled table table-hover table-striped" data-striping="1" id="day-table-2023-09-14" style="font-size: 0.8em">
<caption>2023-09-14 (Jeu)</caption>
<thead>
<tr>
<th>Heure HAE</th>
<th>Niveaux (m)</th>
<th>Niveaux (pi)</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>03:16</td>
<td>4.3</td>
<td>14.2</td>
</tr>
<tr class="even">
<td>09:31</td>
<td>0.7</td>
<td>2.2</td>
</tr>
<tr class="odd">
<td>15:35</td>
<td>4</td>
<td>13</td>
</tr>
<tr class="even">
<td>21:33</td>
<td>0.6</td>
<td>2.1</td>
</tr>
</tbody>
</table>, <table aria-label="Times and Heights for High and Low Tides for 2023-09-15" class="day-tables clearfix table-bordered responsive-enabled table table-hover table-striped" data-striping="1" id="day-table-2023-09-15" style="font-size: 0.8em">
<caption>2023-09-15 (ven)</caption>
<thead>
<tr>
<th>Heure HAE

### Exibindo texto sem formatação, sem tags HTML

In [10]:
print("Seções de texto sem formatação relevantes sem tags HTML:\n")
for parametro_brut in raw_parameters:
    text_view = parametro_brut.getText().strip()
    print(text_view)

Seções de texto sem formatação relevantes sem tags HTML:

2023-09-14 (Jeu)


Heure HAE
Niveaux (m)
Niveaux (pi)




03:16
4.3
14.2


09:31
0.7
2.2


15:35
4
13


21:33
0.6
2.1
2023-09-15 (ven)


Heure HAE
Niveaux (m)
Niveaux (pi)




03:46
4.4
14.4


09:57
0.6
1.9


16:03
4.1
13.5


22:05
0.5
1.6
2023-09-16 (sam)


Heure HAE
Niveaux (m)
Niveaux (pi)




04:16
4.4
14.4


10:24
0.5
1.7


16:31
4.3
13.9


22:37
0.4
1.4
2023-09-17 (dim)


Heure HAE
Niveaux (m)
Niveaux (pi)




04:46
4.3
14.1


10:50
0.5
1.7


16:59
4.3
14.2


23:10
0.4
1.4
2023-09-18 (lun)


Heure HAE
Niveaux (m)
Niveaux (pi)




05:17
4.2
13.7


11:17
0.6
1.9


17:30
4.4
14.3


23:44
0.5
1.7
2023-09-19 (mar)


Heure HAE
Niveaux (m)
Niveaux (pi)




05:50
4
13.1


11:46
0.7
2.3


18:04
4.3
14.2
2023-09-20 (mer)


Heure HAE
Niveaux (m)
Niveaux (pi)




00:20
0.7
2.2


06:26
3.7
12.3


12:17
0.9
2.8


18:42
4.2
13.9


## Renderizando tabelas de marés HTML

* ### Expressões Regulares  `(regex)`

In [11]:
import regex as re
from IPython.display import display, HTML

In [12]:
# Extraindo atributos usando uma expressão regular
FORME_1 = r"(?:.*\n)<caption>(\d\d\d\d-\d\d-\d\d\s*\(\w*\))\<\/caption>"

for parametro_brut in raw_parameters:
    titulo = re.match(re.compile(FORME_1),str(parametro_brut)).group(1)
    substitution = re.sub(FORME_1, '<table>', str(parametro_brut))
    display(HTML(titulo+"\n"+substitution))

Heure HAE,Niveaux (m),Niveaux (pi)
03:16,4.3,14.2
09:31,0.7,2.2
15:35,4,13
21:33,0.6,2.1


Heure HAE,Niveaux (m),Niveaux (pi)
03:46,4.4,14.4
09:57,0.6,1.9
16:03,4.1,13.5
22:05,0.5,1.6


Heure HAE,Niveaux (m),Niveaux (pi)
04:16,4.4,14.4
10:24,0.5,1.7
16:31,4.3,13.9
22:37,0.4,1.4


Heure HAE,Niveaux (m),Niveaux (pi)
04:46,4.3,14.1
10:50,0.5,1.7
16:59,4.3,14.2
23:10,0.4,1.4


Heure HAE,Niveaux (m),Niveaux (pi)
05:17,4.2,13.7
11:17,0.6,1.9
17:30,4.4,14.3
23:44,0.5,1.7


Heure HAE,Niveaux (m),Niveaux (pi)
05:50,4,13.1
11:46,0.7,2.3
18:04,4.3,14.2


Heure HAE,Niveaux (m),Niveaux (pi)
00:20,0.7,2.2
06:26,3.7,12.3
12:17,0.9,2.8
18:42,4.2,13.9


## Extração de dados de marés com expressões regulares

In [13]:
def extrair_numero_station(texto_inserido):
    FORME_1 = r'\s+\((\d*)\)\s+|\s+Pêches\s+et\s+Océans Canada<\/title>'
    return re.search(FORME_1, str(texto_inserido)).group(1)

numero_station = extrair_numero_station(page_resultats)
print("Numéro de estação:",numero_station)

Numéro de estação: 03425


In [14]:
def extrair_fuso_horario(texto_inserido):
    FORME_2 = r'<th>Heure\s*(\w*)<\/th>'
    return re.search(FORME_2, str(texto_inserido)).group(1)

fuso_horario = extrair_fuso_horario(raw_parameters[0])
print("Fuso horario:",fuso_horario)

Fuso horario: HAE


In [18]:
from datetime import datetime

def extrair_data(texto_inserido):
    FORME_3 = r"(\d\d\d\d)-(\d\d)-(\d\d)"
    extraction_resultados = re.search(re.compile(FORME_3),str(texto_inserido))
    # Reescrevendo a data em português
    data_str = extraction_resultados.group(3) + "-" + extraction_resultados.group(2) + "-" + extraction_resultados.group(1) 
    return data_str

extrair_data(raw_parameters[0])

'14-09-2023'

In [19]:
def extrair_dados_mares(texto_inserido):
    FORME_4 = r"<td>(\d*:\d*)<\/td>\n<td>(\d*.\d*)<\/td>"
    extraction_resultados = re.findall(re.compile(FORME_4),str(texto_inserido))
    return extraction_resultados

extrair_dados_mares(raw_parameters[0])

[('03:16', '4.3'), ('09:31', '0.7'), ('15:35', '4'), ('21:33', '0.6')]

## Fazendo backup de dados no arquivo `dados_marés_pag_web.csv`

In [23]:
# Salvando os dados em um arquivo.csv
output_file_path = './'
filename_outpute = "dados_mares_pag_web.csv"
with open(output_file_path+filename_outpute,'w') as arquivo_saida:
    #Escrevendo o cabeçalho do arquivo listando os diferentes atributos
    arquivo_saida.write('localite\tstation\tfuseau_horaire\tdate\theure\thauteur_m\n')
    for parametro_brut in raw_parameters:
        titulo = re.match(re.compile(FORME_1),str(parametro_brut)).group(1)
        data = extrair_data(parametro_brut)
        list_dados_mares = extrair_dados_mares(parametro_brut)
        for dados_mares in list_dados_mares:
            #Savando os dados de marés no arquivo
            arquivo_saida.write(localidade+'\t'+numero_station+'\t'+fuso_horario+'\t'+data+'\t'
                                 +dados_mares[0]+'\t'+dados_mares[1]+'\n')
        
print("Dados de maré salvos no arquivo "+ filename_outpute)

Dados de maré salvos no arquivo dados_mares_pag_web.csv


## Testando a leitura do arquivo de dados

In [24]:
import pandas as pd

filename_outpute = "dados_mares_pag_web.csv"

dados_mares_df = pd.read_csv(output_file_path+filename_outpute,delimiter='\t')
dados_mares_df

,localite,station,fuseau_horaire,date,heure,hauteur_m
0,Tadoussac,3425,HAE,14-09-2023,03:16,4.3
1,Tadoussac,3425,HAE,14-09-2023,09:31,0.7
2,Tadoussac,3425,HAE,14-09-2023,15:35,4.0
3,Tadoussac,3425,HAE,14-09-2023,21:33,0.6
4,Tadoussac,3425,HAE,15-09-2023,03:46,4.4
5,Tadoussac,3425,HAE,15-09-2023,09:57,0.6
6,Tadoussac,3425,HAE,15-09-2023,16:03,4.1
7,Tadoussac,3425,HAE,15-09-2023,22:05,0.5
8,Tadoussac,3425,HAE,16-09-2023,04:16,4.4
9,Tadoussac,3425,HAE,16-09-2023,10:24,0.5
